In [10]:
import json
from typing import List, Dict

import requests
from together import Together
from transformers import AutoTokenizer

# Предобработка входных данных

В данном задании мы будем ходить в онлайн модель. Для получения моего API-ключа пишем мне в лс! Посмотрим, как походы в API соотносятся с тем, что мы делали в практике в шаге "LLM в индустрии".

In [11]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3.1-70B")

## Ручное форматирование промпта

Давайте попробуем собрать вход для llama3.1 руками, для этого допишем функцию `format_messages_to_prompt`.
Она принимает messages - массив словарей, где указаны роли и текст сообщений, а возвращает она текст в формате, который нужно подать модели.

Например для истории сообщений

```python
messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]
```

должен выдаваться итоговый промпт

```text
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>
```

Что важно:
1. Текст начинается со спецтокена bos
2. Дальше идет заголовок start_header_id + end_header_id, которые содержат роль
3. Дальше после \n\n идет текст, заканчивающийся на eot_id
4. Дальше следующий заголовок с новой ролью и т.д.

**Важно** - в данной функции нельзя использовать `tokenizer.apply_chat_template`

In [12]:
def format_messages_to_prompt(messages: List[Dict[str, str]]) -> str:
    result = "<|begin_of_text|>"

    for msg in messages:
        format_msg = f"<|start_header_id|>{msg['role']}<|end_header_id|>\n\n{msg['content']}<|eot_id|>"
        result += format_msg

    return result


messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]



reference_text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>"""


assert format_messages_to_prompt(messages) == reference_text

Мы также помним, что раньше у нас была `tokenizer.apply_chat_template`. Т.к. у нас неофициальный форк llama3.1, то chat_template в токенайзер нам не завезли, поэтому придется добавить его руками

In [13]:
chat_template = """
{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "\n\n"}}
{%- endif %}
{{- "Cutting Knowledge Date: December 2023\n" }}
{{- "Today Date: " + date_string + "\n\n" }}
{%- if tools is not none and not tools_in_user_message %}
    {{- "You have access to the following functions. To call a function, please respond with JSON for a function call." }}
    {{- 'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.' }}
    {{- "Do not use variables.\n\n" }}
    {%- for t in tools %}
        {{- t | tojson(indent=4) }}
        {{- "\n\n" }}
    {%- endfor %}
{%- endif %}
{{- system_message }}
{{- "<|eot_id|>" }}

{#- Custom tools are passed in a user message with some extra guidance #}
{%- if tools_in_user_message and not tools is none %}
    {#- Extract the first user message so we can plug it in here #}
    {%- if messages | length != 0 %}
        {%- set first_user_message = messages[0]['content']|trim %}
        {%- set messages = messages[1:] %}
    {%- else %}
        {{- raise_exception("Cannot put tools in the first user message when there's no first user message!") }}
{%- endif %}
    {{- '<|start_header_id|>user<|end_header_id|>\n\n' -}}
    {{- "Given the following functions, please respond with a JSON for a function call " }}
    {{- "with its proper arguments that best answers the given prompt.\n\n" }}
    {{- 'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.' }}
    {{- "Do not use variables.\n\n" }}
    {%- for t in tools %}
        {{- t | tojson(indent=4) }}
        {{- "\n\n" }}
    {%- endfor %}
    {{- first_user_message + "<|eot_id|>"}}
{%- endif %}

{%- for message in messages %}
    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}
        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' }}
    {%- elif 'tool_calls' in message %}
        {%- if not message.tool_calls|length == 1 %}
            {{- raise_exception("This model only supports single tool-calls at once!") }}
        {%- endif %}
        {%- set tool_call = message.tool_calls[0].function %}
        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}
            {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' -}}
            {{- "<|python_tag|>" + tool_call.name + ".call(" }}
            {%- for arg_name, arg_val in tool_call.arguments | items %}
                {{- arg_name + '="' + arg_val + '"' }}
                {%- if not loop.last %}
                    {{- ", " }}
                {%- endif %}
                {%- endfor %}
            {{- ")" }}
        {%- else  %}
            {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' -}}
            {{- '{"name": "' + tool_call.name + '", ' }}
            {{- '"parameters": ' }}
            {{- tool_call.arguments | tojson }}
            {{- "}" }}
        {%- endif %}
        {%- if builtin_tools is defined %}
            {#- This means we're in ipython mode #}
            {{- "<|eom_id|>" }}
        {%- else %}
            {{- "<|eot_id|>" }}
        {%- endif %}
    {%- elif message.role == "tool" or message.role == "ipython" %}
        {{- "<|start_header_id|>ipython<|end_header_id|>\n\n" }}
        {%- if message.content is mapping or message.content is iterable %}
            {{- message.content | tojson }}
        {%- else %}
            {{- message.content }}
        {%- endif %}
        {{- "<|eot_id|>" }}
    {%- endif %}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
{%- endif %}
""".strip()
tokenizer.chat_template = chat_template

## Автоматическая сборка промпта

Давайте вспомним теперь на деле, как используется chat_template! Попробуем использовать функцию `tokenizer.apply_chat_template`

In [14]:
messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False)

In [15]:
reference_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>"""

assert prompt == reference_prompt

Обратите внимание, что в заданном chat_template указаны Cutting Knowledge Date, т.е. до данные до какого периода видела модели, и Today Date - захардкоженная дата текущего диалога.

Подумайте, на что влияет аргумент `add_generation_prompt` в функции `tokenizer.apply_chat_template`? Зачем его использовать?

## Походы в API

Теперь давайте посмотрим, как можно ходить в API. Вообще говоря различных провайдеров много, API у них у всех очень похожий, т.к. все мимикрируют под OpenAI. Собственно, за апишкой – ко мне!

In [3]:
# Вставьте свой ключ из https://api.together.ai/
API_KEY = "tgp_v1_gw1i6zpF-X3Ptq5aSuzMhlu0ATGk0lhuIhABsa46Obo"

Есть несколько способов сходить в API. Можно ходить напрямую через библиотеку **requests**. Допишите post запрос в `url` с данными `data` и заголовками `headers`.

In [17]:
headers = {
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json',
}
url = "https://api.together.xyz/v1/chat/completions"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "messages": messages
}

response = requests.post(url, headers=headers, json=data)
model_answer = response.json()["choices"][0]["message"]["content"]
assert "london" in model_answer.lower()

Мы подали messages, дальше они каким-то образом собрались в promt и подались модели. Мы не знаем, какой промпт используется на стороне провайдера. Вспомним про Today Date из предыдущего пункта задания - использует ли его together? Обновляют ли они его сегодняшним днем или оставляют Today Date? Если обновляют, то по какому часовому поясу?

Чтобы ответы на эти и многие другие вопросы не мучали нас по ночам, можно использовать prompt формат, а именно подать модели текст напрямую на генерацию. Давайте для этого используем `tokenizer.apply_chat_template`. Модель будет принимать текст ровно так, как вы его подадите, без каких-либо предобработок. Подумайте, нужно ли вам использовать аргумент `add_generation_prompt`?

Чтобы послать запрос напрямую, нужно в предыдущем запросе убрать messages, который представляет из себя список словарей, и послать поле prompt - строку с промптом для модели.

In [18]:
headers = {
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json',
}
url = "https://api.together.xyz/v1/chat/completions"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "prompt": prompt
}

response = requests.post(url, headers=headers, json=data)
model_answer = response.json()["choices"][0]["message"]["content"]
assert "london" in model_answer.lower() and "assistant" not in model_answer.lower()

## Клиент

Теперь мы понимаем общую схему взаимодействия с провайдером - они предоставляют апи, куда можно посылать или промтп или историю диалога. При посылке промпта вся ответственность за формат ложится на нас, при посылке messages форматтинг происходит на стороне провайдера, но мы не всегда представляем, как он работает. Выбор в пользу того или иного варианта всегда остается на вас.

Мы использовали выше библиотеку requests, чтобы послать HTTP-запрос на сервера together, однако есть способ и проще - python client. Давайте познакомимся с ним поближе. Для этого давайте используем функцию `client.chat.completions.create`. Также давайте добавим опции сэмплинга, которые в этой функции поддержаны. Их можно посылать и в запросах через requests, но мы здесь и далее будем пользоваться клиентом.
* top_k = 100
* temperature = 0.5
* top_p = 0.9
* repetition_penalty = 1.05

In [19]:
client = Together(api_key=API_KEY)

In [20]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    top_k=100,
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.05,
    max_tokens=64
)

response_text = response.choices[0].message.content
assert "london" in response_text.lower()

Аналогично посылать просто prompt можно через `client.completions.create`.

## Tools

Давайте теперь посмотрим, как можно использовать tools в связке с моделями. У нас есть функция, которая входит в базу данных и получает информацию о юзере. Базы данных, конечно же, у нас никакой нет, но у нас есть некоторая функция, которая эмулирует это поведение, так что давайте попробуем ее описать.


In [21]:
def get_user_info_from_db(person_name: str) -> Dict[str, str]:
    database = {
        "ilya": {
            "job": "Software Developer",
            "pets": "dog",
        },
        "farruh": {
            "job": "Senior Data & Solution Architect",
            "hobby": "travelling, hiking",
        },
        "timur": {
            "job": "DeepSchool Founder",
            "city": "Novosibirsk",
        }
    }
    no_info = {"err": f"No info about {person_name}"}
    return database.get(person_name.lower(), no_info)

print(get_user_info_from_db("Timur"))

{'job': 'DeepSchool Founder', 'city': 'Novosibirsk'}


Давайте попробуем описать эту функцию в формате json, чтобы модель могла ее увидеть!
Заполните поля в определении дальше

In [22]:
get_user_info_from_db_tool = {
    "type": "function",
    "function": {
        "name": "get_user_info_from_db",
        "description": "Fetches information about a person from a simulated database and returns a dictionary of known attributes (e.g., job, pets, hobby, city). If the person is not found, returns an error dict.", 
        "parameters": {
            "type": "object",
            "properties": {
                "person_name": {
                    "type": "string",
                    "description":"Name of the person to look up in the database (case-insensitive)."
                }
            },
            "required": ["person_name"] 
        }
    }
}

Теперь давайте подадим это описание в `tokenizer.apply_chat_template`. Обратите внимание на его аргумент `tools`! Не забудьте `add_generation_prompt`, если он нужен.

In [26]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
prompt = tokenizer.apply_chat_template(
    messages,
    tools=[get_user_info_from_db_tool],
    tokenize=False,
    add_generation_prompt=True
)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "get_user_info_from_db",
        "description": "Fetches information about a person from a simulated database and returns a dictionary of known attributes (e.g., job, pets, hobby, city). If the person is not found, returns an error dict.",
        "parameters": {
            "type": "object",
            "properties": {
                "person_name": {
                    "type": "string",
                    "description": "Name of the person t

Давайте пошлем наш запрос в модель. На выбор 2 модели, если не будет работать с 8b, то предлагается посылать в 70b.
Для данного запроса для 8b был подобран работающий `seed=9706540181089681000`, который можно подать в функцию.

Давайте воспользуемся `client.completions.create` для генерации ответа от модели.

In [4]:
model_8b = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
model_70b = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"

In [27]:
response_8b = client.completions.create(
    model=model_8b,
    prompt=prompt,
    seed=9706540181089681000,
    max_tokens=64
)
response_70b = client.completions.create(
    model=model_70b,
    prompt=prompt,
    max_tokens=64
)

print(response_8b.choices[0].text)
print(response_70b.choices[0].text)

{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}
{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}


Если все хорошо, то мы получили ответ от модели, который выглядит как некоторый структурированный вывод, который можно использовать для вызова модели. Давайте попробуем написать функцию, которая принимает ответ модели в "сыром виде", выбирает, какую функцию с какими аргументами вызвать.

Здесь нам поможет FUNCTION_REGISTRY и то, что параметры в функцию можно передавать как словарь, например так
```python
def foo(a, b, c):
    print(a, b, c)

obj = {'b':10, 'c':'lee'}

foo(100, **obj)
```

In [32]:
FUNCTION_REGISTRY = {"get_user_info_from_db": get_user_info_from_db}
# На случай, если модель не генерит function call
reference_answer = """{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}"""


def parse_function_call(model_answer):
    s = model_answer.strip()

    try:
        payload = json.loads(s)
    except json.JSONDecodeError:
        return None
    
    name = payload.get("name")
    params = payload.get("parameters")

    fn = FUNCTION_REGISTRY.get(name)

    return fn(**params)


assert parse_function_call(reference_answer) == get_user_info_from_db("Ilya")

Теперь давайте попробуем объединить все это в историю диалога и сгенерировать моделью финальный ответ.
Для этого в messages, где хранится наша история диалога нужно добавить
1. Вызов function call моделью с ролью ХХХ (это часть задания, напишите сами)
2. Ответ function call с ролью tool

После этого данный промпт нужно послать модели снова, чтобы получить финальный ответ.
Для этого опять используем `tokenizer.apply_chat_template` и `client.completions.create`.

В зависимости от модели может понадобиться убрать tools (на 8b, 70b должна справиться). Для 8b опять же подобран seed=2017684582943914000

In [33]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]

prompt1 = tokenizer.apply_chat_template(
    messages,
    tools=[get_user_info_from_db_tool],
    tokenize=False,
    add_generation_prompt=True
)
resp1 = response_8b = client.completions.create(
    model=model_8b,
    prompt=prompt,
    seed=9706540181089681000,
    max_tokens=64
)
ans1 = resp1.choices[0].text
messages.append({"role": "assistant", "content": ans1})

tool_output = parse_function_call(ans1)
messages.append({"role": "tool", "content": tool_output})
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(prompt)

response_8b = client.completions.create(model=model_70b, prompt=prompt, seed=2017684582943914000)
print(response_8b.choices[0].text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

What do you know about Ilya?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}<|eot_id|><|start_header_id|>ipython<|end_header_id|>

{"job": "Software Developer", "pets": "dog"}<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Ilya is a software developer and has a dog.


Теперь давайте посмотрим на chat-API, как обрабатываются function calls там?
Используем для этого уже знакомый `client.chat.completions.create`, обратим внимание на аргумент tools внутри него. Здесь рекомендуется использовать 70b модель. На всякий случай работающий seed=14157400267283583000

In [34]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
response = client.chat.completions.create(
    model=model_70b,
    messages=messages,
    tools=[get_user_info_from_db_tool],
    seed=14157400267283583000
)

Мы можем видеть, что у нас не работает предыдущий подход с полем `content`, однако должно было появиться поле `tool_calls`, которое содержит в себе информацию о вызове инструмента

In [35]:
response.choices[0].message.tool_calls

[ToolCalls(id='call_p1fouj4ylwkg12frb0qpt8he', type='function', function=FunctionCall(name='get_user_info_from_db', arguments='{"person_name":"Ilya"}'), index=0)]

# Использование библиотек

Теперь, когда мы руками прошли весь пути обработки function call можно посмотреть уже на готовые инструменты.
Мы много чего сделали руками:
1. Писали описание функции
2. Обрабатывали ответ
3. Вызывали функцию
4. Возвращали все это в модель

Давайте теперь посмотрим, как оно работает в библиотеках!

**NB** - библиотеки развиваются и вполне, возможно, что к концу роадмапа те интерфейсы, которые мы используем в этом домашнем задании будут уже неактуальны, но я уверен, что знаний и принципов, полученных из этих заданий хватит, чтобы адаптироваться к будущим вызовам!

In [ ]:
# ! pip install langchain==0.2.16 llama-index-core==v0.11.16 langchain-together==0.2.0 llama-index-llms-together==0.2.0

# LangChain

In [ ]:
import os
from langchain_together import ChatTogether
from langchain_core.tools import tool

Давайте ознакомимся с langchain-интеграцией together.ai

In [3]:
os.environ["TOGETHER_API_KEY"] = API_KEY
model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"

llm = ChatTogether(model=model)

In [4]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
response = llm.invoke(messages)
print(response.content)

There are several notable individuals and entities named Ilya. Here are a few:

1. Ilya Muromets: Ilya Muromets is a legendary hero in Russian folklore. He was a medieval warrior and a prince from the city of Murom. According to legend, he was a skilled fighter and a wise leader who fought against the Mongols and other invaders.

2. Ilya Kovalchuk: Ilya Kovalchuk is a Russian professional ice hockey player who played in the National Hockey League (NHL) for the Atlanta Thrashers, New Jersey Devils, and Los Angeles Kings. He is a highly skilled forward and one of the top scorers in NHL history.

3. Ilya Prigogine: Ilya Prigogine was a Russian-born Belgian chemist and Nobel laureate who made significant contributions to the field of thermodynamics and the study of complex systems. He was awarded the Nobel Prize in Chemistry in 1977 for his work on the theory of dissipative structures.

4. Ilya Repin: Ilya Repin was a Russian painter who is considered one of the most important figures in R

Теперь, когда мы разобрались, как базово работать с langchain, давайте попробуем добавить инструментов. Чтобы нам было не так скучно, давайте напишем новую функцию, которая считает "волшебную операцию".

Эта функция принимает 2 строки, возвращает строку строку b в обратном порядке, сконкатенированную со строкой a. Допишите эту функцию.

In [7]:
def magic_operation(a, b):
    return b[::-1] + a

assert magic_operation("456", "321") == "123456"

Теперь давайте обернем эту функцию в декоратор tool из langchain, аннотируем типы и допишем docstring. После этого можно будет автоматически сгенерировать описани функции в function call формате!

In [9]:
@tool
def magic_operation_tool(a: str, b: str) -> str:
    """
    Concatenate the reverse of `b` with `a`.

    Args:
        a: First string (will be appended to the end).
        b: Second string (will be reversed and placed at the beginning).

    Returns:
        A new string equal to reverse(b) + a.
    """
    return magic_operation(a, b)

print(magic_operation_tool.args_schema.schema())

{'description': 'Concatenate the reverse of `b` with `a`.\n\nArgs:\n    a: First string (will be appended to the end).\n    b: Second string (will be reversed and placed at the beginning).\n\nReturns:\n    A new string equal to reverse(b) + a.', 'properties': {'a': {'title': 'A', 'type': 'string'}, 'b': {'title': 'B', 'type': 'string'}}, 'required': ['a', 'b'], 'title': 'magic_operation_tool', 'type': 'object'}


C:\Users\Dmitry\AppData\Local\Temp\ipykernel_22660\1743887400.py:15: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(magic_operation_tool.args_schema.schema())


Теперь давайте попробуем подать запрос в нашу LLM и обогатить ее нашим function_call. Для этого нужна функция `llm.bind_tools`.

In [10]:
llm_with_tools = llm.bind_tools([magic_operation_tool])

Теперь давайте как и раньше:
1. Сгенерируем ответ на messages
2. Проверим в ответе resp.tool_calls, вызовем нужный инструмент
3. Расширим messages ответом модели и ответом инструмента, сгенерируем финальный ответ.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`"}
]
resp = llm_with_tools.invoke(messages)

if resp.tool_calls:
    messages.append(resp)

    tool_out = magic_operation_tool.invoke(resp.tool_calls[0])
    messages.append(tool_out)

In [14]:
assert len(messages) == 4

In [15]:
res = llm.invoke(messages).content
assert "123456" in res

# LlamaIndex

Аналогичный инструмент LlamaIndex. В ней не так хороша поддержка function calls не для OpenAI, поэтому придется забежать вперед и использовать ReActAgent.

In [ ]:
from llama_index.llms.together import TogetherLLM
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

In [5]:
llm = TogetherLLM(model=model_70b, api_key=API_KEY)

Скопируйте magic_operation_tool из части с langchain сюда,  но без декоратора.

In [6]:
def magic_operation_tool(a: str, b: str) -> str:
    """
    Concatenate the reverse of `b` with `a`.

    Args:
        a: First string (will be appended to the end).
        b: Second string (will be reversed and placed at the beginning).

    Returns:
        A new string equal to reverse(b) + a.
    """
    print("INSIDE FUNCTION CALL")
    return magic_operation(a, b)

Мы можем аналогично создать инструмент с помощью `FunctionTool.from_defaults`

In [8]:
magic_operation_tool_llamaindex = FunctionTool.from_defaults(fn=magic_operation_tool)
print(magic_operation_tool_llamaindex.metadata)

ToolMetadata(description='magic_operation_tool(a: str, b: str) -> str\n\n    Concatenate the reverse of `b` with `a`.\n\n    Args:\n        a: First string (will be appended to the end).\n        b: Second string (will be reversed and placed at the beginning).\n\n    Returns:\n        A new string equal to reverse(b) + a.', name='magic_operation_tool', fn_schema=<class 'llama_index.core.tools.utils.magic_operation_tool'>, return_direct=False)


Давайте создадим ReActAgent: ему нужно передать tools, llm, memory=None и verbose=True

In [9]:
agent = ReActAgent(
    tools=[magic_operation_tool_llamaindex],
    llm=llm,
    memory=None,
    verbose=True
)

In [13]:
text = "Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`"
handler = agent.run(text)
response = await handler

INSIDE FUNCTION CALL


In [14]:
str(response)

"The result of the magic operation for strings '456' and '321' is 123456."

# Agents

Настала пора сделать своего агента!
Попробуем сделать финансового аналитика. Требования следующие:
бот должен по запросу данных о какой-либо компании смотреть самые большие изменения цены ее акций за последний месяц, после чего бот должен объяснить, с какой новостью это связано.

Предлагается не строить сложную систему с классификаторами, а отдать всю сложную работу агенту. Давайте посмотрим, какие API нам доступны.

Первым делом получение котировок - для этого нам поможет библиотека yfinance. По названию компании и периоду отчетности можно посмотреть открывающие цены на момент открытия и закрытия биржи.

In [1]:
import yfinance as yf

stock = yf.Ticker("AAPL") # посмотрим котировки APPLE
df = stock.history(period="1mo")
df[["Open", "Close"]]

,Open,Close
Date,,
2025-12-29 00:00:00-05:00,272.690002,273.760010
2025-12-30 00:00:00-05:00,272.809998,273.079987
2025-12-31 00:00:00-05:00,273.059998,271.859985
2026-01-02 00:00:00-05:00,272.260010,271.010010
2026-01-05 00:00:00-05:00,270.640015,267.260010
2026-01-06 00:00:00-05:00,267.000000,262.359985
2026-01-07 00:00:00-05:00,263.200012,260.329987
2026-01-08 00:00:00-05:00,257.019989,259.040009
2026-01-09 00:00:00-05:00,259.079987,259.369995


Для поиска новостей нам поможет https://newsapi.org/
Можно легко получить свой ключ за короткую регистрацию, дается 1000 запросов в день, каждый запрос может включать в себя ключевое слово и промежуток дат. По бесплатному апи ключу дается ровно 1 месяц, что нам подходит.

In [2]:
import requests

api_key = "90b6dcd60c4049ac8cc5293002e95020" 
api_template = "https://newsapi.org/v2/everything?q={keyword}&apiKey={api_key}&from={date_from}"

articles = requests.get(api_template.format(keyword="Apple", api_key=api_key, date_from="2026-01-01")).json()

for article in articles["articles"]:
    if article["title"] != "[Removed]":
        print(article["title"])
        print(article["description"])
        break

What Apple and Google’s Gemini deal means for both companies
For years, Apple and Google have had a will-they-won't-they type of relationship, as far as which AI company Apple would pick to underpin its Siri virtual assistant and give it new AI-fueled personalization and agentic capabilities. Apple has spent the last y…


Очень много статей заблокированы и имеют название `[Removed]`, нужно их отфильтровать. В оставшихся статьях будем брать только title (заголовок) и description (описание или краткий пересказ).

Вам необходимо реализовать [ReAct Agent](https://react-lm.github.io/). Особенность этого агента заключается в том, что он вначале формирует мысль, а потом вызывает действие (function call) для достижения какой-либо цели.

Что нужно сделать:
1. Описать и реализовать function call для определения, в какой день была самая большая разница в цене акций в момент открытия и закрытия биржи. Функция получает один аргумент - название акций компании (например AAPL для Apple), а выдает словарь с 2мя полями: с датой максимальной разницы в ценах и самой разницей в ценах.
2. Описать и реализовать function call для получения 5 релевантных новостей о компании. В качестве аргумента принимаются название компании и дата. Ваша задача - сходить в newsapi, получить новости и вернуть 5 случайных новостей, которые произошли не позже чем день торгов. Если новостей меньше 5, то верните столько, сколько получится.
3. После этого агент должен вернуть ответ, в котором постарается аргументировать изменения в цене.


Реализовывать агента можно любым удобным способом, в том числе взять готовые имплементации.
1. [LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/agent/react_agent/) - вдобавок можно посмотреть предыдущее задание, где он уже используется.
2. [Langchain/Langgraph](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/#code)
3. Написать полностью свою реализацию


Не забудьте, что очень важно описать задачу в промпте: нужно сказать, какие цели у агента и что он должен сделать. У функций должны быть говорящие описания, чтобы LLM без лишних проблем поняла, какие есть функции и когда их использовать. По всем вопросам можно обращаться в наш телеграм-чат в канал "Tools & Agents".


In [3]:
import yfinance as yf


def get_max_open_close_diff(ticker: str) -> dict:
    """
    Находит день за последний месяц, когда абсолютная разница |Open - Close| была максимальной.
    
    Args:
        ticker: тикер акции (например "AAPL")

    Returns:
        dict: {"date": "YYYY-MM-DD", "diff": float}
    """
    stock = yf.Ticker(ticker)
    df = stock.history(period="1mo")

    diff_series = (df["Open"] - df["Close"]).abs()

    max_ts = diff_series.idxmax()
    max_diff = float(diff_series.loc[max_ts])

    max_date = max_ts.date().isoformat()

    return {"date": max_date, "diff": max_diff}

In [ ]:
import os
import random

import requests

os.environ["NEWSAPI_KEY"] = "90b6dcd60c4049ac8cc5293002e95020" 

def get_company_news(company: str, trade_day: str, k: int = 5) -> list[dict]:
    """
    Получить до k случайных релевантных новостей по компании, опубликованных не позже trade_day.

    Args:
        company: название компании (например "Apple")
        trade_day: дата торгов в формате "YYYY-MM-DD"
        k: сколько новостей вернуть (по умолчанию 5)

    Returns:
        list[dict]: список новостей (title, description)
    """
    api_key = os.getenv("NEWSAPI_KEY")
    api_template = "https://newsapi.org/v2/everything?q={keyword}&apiKey={api_key}&from={date_from}"

    articles = requests.get(api_template.format(keyword=company, api_key=api_key, date_from=trade_day)).json()

    filtered = []
    for article in articles["articles"]:
        if article["title"] != "[Removed]":
            filtered.append({"title": article["title"], "description": article["description"]})

    random.shuffle(filtered)
    return filtered[:k]

In [5]:
from langchain.tools import tool

@tool
def tool_get_max_open_close_diff(ticker: str) -> dict:
    """Find date in last 1mo where abs(Open-Close) is maximal. Input: ticker like AAPL. Output: {date, diff}."""
    return get_max_open_close_diff(ticker)

@tool
def tool_get_company_news(company: str, trade_day: str, k: int = 5) -> list[dict]:
    """Get up to 5 random relevant news about company published not later than trade_day (YYYY-MM-DD)."""
    return get_company_news(company, trade_day, k)


TOOLS = [tool_get_max_open_close_diff, tool_get_company_news]

In [6]:
from langchain_together import ChatTogether

os.environ["TOGETHER_API_KEY"] = "tgp_v1_gw1i6zpF-X3Ptq5aSuzMhlu0ATGk0lhuIhABsa46Obo"

llm = ChatTogether(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

c:\Users\Dmitry\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
SYSTEM_PROMPT = """Ты финансовый аналитик.
Твоя задача: по запросу пользователя про компанию определить, в какой день за последний месяц была максимальная разница между ценой открытия и закрытия акции, а затем объяснить, с какой новостью это связано.

Правила:
1) Сначала определи тикер акции (например, AAPL) и название компании (например, Apple) из запроса пользователя.
   Если компания не указана явно, используй тикер как keyword для новостей.
2) Вызови tool_get_max_open_close_diff(ticker) и получи {date, diff}.
3) Затем вызови tool_get_company_news(company, trade_day) с датой из шага 2.
4) Верни ответ по-русски:
   - дата и величина движения (Open vs Close),
   - 1–3 самых релевантных заголовка,
   - объяснение вероятной связи новости и движения цены.
Если новостей нет — честно скажи это и предложи возможные причины (рынок/сектор/отчетность).
Не показывай внутренние размышления (Thought/Chain-of-thought), только итоговый ответ.
"""

In [8]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent = create_react_agent(
    model=llm,
    tools=TOOLS,
    state_modifier=SYSTEM_PROMPT,
    checkpointer=memory
)

In [9]:
result = agent.invoke(
    input={"messages": [("user", "Проанализируй AAPL (Apple)")]},
    config={"configurable": {"thread_id": "demo-1"}}
)

result

{'messages': [HumanMessage(content='Проанализируй AAPL (Apple)', additional_kwargs={}, response_metadata={}, id='2a19f565-0955-4bcc-b8cf-25f145cdb4ec'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_v8yj739zp5qn4mdtohwq0wg6', 'function': {'arguments': '{"ticker":"AAPL"}', 'name': 'tool_get_max_open_close_diff'}, 'type': 'function', 'index': 0}, {'id': 'call_cktwa1tdleuhulwnofwdwou0', 'function': {'arguments': '{"company":"Apple","trade_day":"2024-01-28","k":3}', 'name': 'tool_get_company_news'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 626, 'total_tokens': 686, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'cached_tokens': 512}, 'model_name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', 'system_fingerprint': None, 'id': 'oVDboGF-2j9zxn-9c57d852ca6fec20', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--019c092c-6cfa-7113-

In [11]:
print(result["messages"][-1].content)

Дата и величина движения: 20 января 2026 года, Open vs Close: 6,029998779296875.

1. Apple va a romper todos los récords con su nuevo MacBook Pro: su nuevo chip M5 Max va a ser una locura.
2. Why is the Apple Weather App on the iPhone Predicting So Much Snow?
3. Apple Music is sneakily becoming the best music streamer for Android.

Вероятная связь новости и движения цены: Движение цены Apple на 20 января 2026 года может быть связано с новостями о новом MacBook Pro с чипом M5 Max, который может быть анонсирован в этом году. Это может привести к повышению цены акций Apple из-за ожиданий отрасли по поводу нового продукта.
